<a href="https://colab.research.google.com/github/atulghunavat/atul_INFO5731_Spring2020/blob/master/In_class_exercise/In_class_exercise_04_Atul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise**

# 1. Text Data Preprocessing

Here is a [legal case](https://github.com/unt-iialab/INFO5731_Spring2020/blob/master/In_class_exercise/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



## 1.1 Basic feature extraction using text data

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

## 1.2 Basic Text Pre-processing of text data

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

## 1.3 Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above.


## 1.4 Advance Text Processing

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 2-grams terms as features.


In [375]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import requests
page = 'https://raw.githubusercontent.com/unt-iialab/INFO5731_Spring2020/master/In_class_exercise/01-05-1%20%20Adams%20v%20Tanner.txt'
data = requests.get(page).text
# print (page.text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [376]:
#1.1
number_of_sentences = sent_tokenize(data)
number_of_words = word_tokenize(data)
characters = len(data)
print('Number of sentences: ', len(number_of_sentences))
print('Number of words: ', len(number_of_words))
print('Number of characters: ', characters)

w_avg = sum(len(word) for word in number_of_words)/len(number_of_words)
print ('Average word length: ', w_avg)

stop = stopwords.words('english')
stopwords = len([x for x in data.split() if x in stop])
print('Number of stopwords: ', stopwords)

regex = re.compile('[^a-zA-Z\d\s]')
special = 0
for sent in data.split():
  chars = regex.finditer(sent)
  for char in chars:
    special += 1
print('Number of special: ', special)

numeric = len([x for x in data.split() if x.isdigit()])
print('Number of numeric: ', numeric)

upper = len([x for x in data.split() if x.isupper()])
print('Number of uppercase: ', upper)

Number of sentences:  173
Number of words:  4368
Number of characters:  20613
Average word length:  3.827838827838828
Number of stopwords:  1679
Number of special:  817
Number of numeric:  60
Number of uppercase:  84


In [377]:
#1.2 & 1.4
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from textblob import TextBlob

raw_data = data.split("\n")
cleaned_df = pd.DataFrame()
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def remove_punctuation(text):
  rm_punc = "".join([char for char in text if char not in string.punctuation])
  return rm_punc

def remove_stopwords(text):
  words = " ".join([word for word in text.split() if word not in stopwords.words('english')])
  return words

def find_frequentword(word_list):
  freq = (pd.Series(word_list)).value_counts()[:10]
  return freq

def remove_frequentwords(text, f_word_list):
  words = " ".join([word for word in text.split() if word not in f_word_list])
  return words

def find_rareword(word_list):
  freq = (pd.Series(word_list)).value_counts()[-10:]
  return freq

def remove_rarewords(text, r_word_list):
  words = " ".join([word for word in text.split() if word not in r_word_list])
  return words

def perform_stemming(text):
  stem = " ".join([stemmer.stem(i) for i in text.split()])
  return stem

def perform_lemmatization(text):
  lem = " ".join([lemmatizer.lemmatize(i) for i in text.split()])
  return lem

cleaned_array = []
for sentence in raw_data:
  clean_sent = sentence.lower()
  clean_sent = remove_punctuation(clean_sent)
  clean_sent = remove_stopwords(clean_sent)
  if(len(clean_sent) > 0):
    cleaned_array.append(clean_sent)

word_list = []
for i in cleaned_array:
  for j in i.split():
    word_list.append(j)
frequent_word_list = find_frequentword(word_list)
rare_word_list = find_rareword(word_list)

new_cleaned_array = []
for sentence in cleaned_array:
  clean_sent = remove_frequentwords(sentence, frequent_word_list)
  clean_sent = remove_rarewords(clean_sent, rare_word_list)
  clean_sent = TextBlob(clean_sent).correct()
  clean_sent = " ".join(clean_sent.words)
  clean_sent = perform_stemming(clean_sent)
  clean_sent = perform_lemmatization(clean_sent)
  if(len(clean_sent) > 0):
    new_cleaned_array.append(clean_sent)

cleaned_df['Cleaned'] = new_cleaned_array

cleaned_df.to_csv('/content/legal.csv')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [378]:
#1.4
from textblob import TextBlob

ngram_2 = []
ngram_1 = []

for word in cleaned_df['Cleaned']:
  ngram2 = TextBlob(word).ngrams(2)
  if(len(ngram2) > 0):
      ngram_2.append(ngram2)
  ngram1 = TextBlob(word).ngrams(1)
  if(len(ngram1) > 0):
      ngram_1.append(ngram1)

print('Top 10 1 gram:')

ngram_1.sort(key = len) 
ngram_1.reverse()
for n in ngram_1[:10]:
    print('Total Items:' + str(len(n)))
    print(n)

print('Top 10 2 gram:')

ngram_2.sort(key = len) 
ngram_2.reverse()
for n in (ngram_2[:10]):
    print('Total Items:' + str(len(n)))
    print(n)


Top 10 1 gram:
Total Items:310
[WordList(['3']), WordList(['bring']), WordList(['u']), WordList(['back']), WordList(['question']), WordList(['whether']), WordList(['plaintiff']), WordList(['defeat']), WordList(['mortgag']), WordList(['frequent']), WordList(['loot']), WordList(['whether']), WordList(['common']), WordList(['law']), WordList(['corn']), WordList(['c']), WordList(['gather']), WordList(['seiz']), WordList(['fieri']), WordList(['facial']), WordList(['mr']), WordList(['dane']), WordList(['remark']), WordList(['upon']), WordList(['point']), WordList(['say']), WordList(['“']), WordList(['the']), WordList(['american']), WordList(['editor']), WordList(['bacon']), WordList(['’']), WordList(['s']), WordList(['abridg']), WordList(['say']), WordList(['‘']), WordList(['wheat']), WordList(['ground']), WordList(['chattel']), WordList(['subject']), WordList(['taken']), WordList(['sheriff']), WordList(['may']), WordList(['suffer']), WordList(['grow']), WordList(['till']), WordList(['harves

# 2. Python Regular Expression

[link text](https://)## 2.1 Write a Python program to remove leading zeros from an IP address. 

ip = "260.08.094.109"

In [379]:
# 2.1
import re
ip = "260.08.094.109"
clean_ip = re.sub('\.[0]*', '.', ip)
print(clean_ip)

260.8.94.109


## 2.2 Write a Python Program to extract all the years from the following sentence.

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [380]:
# 2.2
import re
sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."
years = re.findall(r"[\d]{4}", sentence)
year = []

for num in years:
  if (num.startswith('20')):
      year.append(num)

print(year)


['2010', '2010', '2019']
